<a href="https://colab.research.google.com/github/chottokun/spaCy_training/blob/main/Spacy_with_tf_learning01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spaCyで東北大のBERTモデルでNER学習をする。

まずは、こちらを参考に・・・  
https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part15.html

configがうまく行かないので  
https://github.com/megagonlabs/UD_Japanese-GSD/blob/master/spacy/config/ja_gsd_bert_v2.cfg  
こちらを参考に作り直した。  

先人に感謝  
https://github.com/megagonlabs/UD_Japanese-GSD/blob/master/leader_board.md  



In [1]:
#in colab cell
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
!pip install -U spacy[cuda112,transformers,lookups,ja]
!apt-get install mecab mecab-ipadic-utf8
!pip install mecab-python3 fugashi ipadic
!pip install unidic-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77.5 MB 1.6 MB/s 
     |████████████████████████████████| 2.2 MB 51.3 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 98.5 MB 83 kB/s 
     |████████████████████████████████| 4.7 MB 54.0 MB/s 
     |████████████████████████████████| 1.1 MB 67.1 MB/s 
     |████████████████████████████████| 163 kB 90.3 MB/s 
     |████████████████████████████████| 6.6 MB 80.0 MB/s 
  Created wheel for sudachidict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=0e69241b6cb9c5a1398789072cb7006440172af13fae6e65a2e2027662609910
  Stored in directory: /root/.cache/pip/wheels/66/a9/e1/bde612c31f0ae6877e7e39f278076befd399c488cba80292b6
Successfully built sudachidict-core
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was

In [3]:
!git clone -b feature/bert_base_japanese_v2 https://github.com/megagonlabs/UD_Japanese-GSD

Cloning into 'UD_Japanese-GSD'...
remote: Enumerating objects: 693, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 693 (delta 7), reused 13 (delta 7), pack-reused 675
Receiving objects: 100% (693/693), 73.31 MiB | 20.65 MiB/s, done.
Resolving deltas: 100% (403/403), done.


In [4]:
!mkdir -p corpus
!python -m spacy convert ./UD_Japanese-GSD/ja_gsd-ud-train.ne.conllu ./corpus
!python -m spacy convert ./UD_Japanese-GSD/ja_gsd-ud-dev.ne.conllu ./corpus
!python -m spacy convert ./UD_Japanese-GSD/ja_gsd-ud-test.ne.conllu ./corpus

!ls ./corpus

ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (7050 documents):
corpus/ja_gsd-ud-train.ne.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (507 documents):
corpus/ja_gsd-ud-dev.ne.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (543 documents):
corpus/ja_gsd-ud-test.ne.spacy
ja_gsd-ud-dev.ne.spacy	ja_gsd-ud-test.ne.spacy  ja_gsd-ud-train.ne.spacy


In [5]:
import spacy
from spacy.tokens import Doc, DocBin

nlp = spacy.blank("ja") 

train_data = DocBin()
train_data.from_disk(path="./corpus/ja_gsd-ud-train.ne.spacy")

docs = list(train_data.get_docs(nlp.vocab))

for token in docs[2]:
  print("text=%s head=%d dep=%s ent_type=%s ent_job=%s" % (token, token.head.i, token.dep_, token.ent_type_, token.ent_iob_))

text=手 head=2 dep=obl ent_type= ent_job=O
text=に head=0 dep=case ent_type= ent_job=O
text=持っ head=6 dep=acl ent_type= ent_job=O
text=た head=2 dep=aux ent_type= ent_job=O
text=特殊 head=6 dep=acl ent_type= ent_job=O
text=な head=4 dep=aux ent_type= ent_job=O
text=刃物 head=8 dep=obj ent_type= ent_job=O
text=を head=6 dep=case ent_type= ent_job=O
text=使っ head=12 dep=acl ent_type= ent_job=O
text=た head=8 dep=aux ent_type= ent_job=O
text=アクロバティック head=12 dep=acl ent_type= ent_job=O
text=な head=10 dep=aux ent_type= ent_job=O
text=体術 head=22 dep=nmod ent_type= ent_job=O
text=や head=12 dep=case ent_type= ent_job=O
text=、 head=12 dep=punct ent_type= ent_job=O
text=揚羽 head=18 dep=obl ent_type=PERSON ent_job=B
text=と head=15 dep=case ent_type= ent_job=O
text=薄羽 head=18 dep=compound ent_type=PERSON ent_job=B
text=同様 head=24 dep=advcl ent_type= ent_job=O
text=に head=18 dep=aux ent_type= ent_job=O
text=クナイ head=22 dep=nmod ent_type= ent_job=O
text=や head=20 dep=case ent_type= ent_job=O
text=忍具 head=24 de

In [6]:
%%bash
cat << EOF > base_config.cfg
[paths]
train = null                      
dev = null                 
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "ja"
pipeline = ["transformer","parser","ner", "morphologizer"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null

[nlp.tokenizer]
@tokenizers = "spacy.ja.JapaneseTokenizer"
split_mode = "A"

[components]

[components.morphologizer]
factory = "morphologizer"

[components.morphologizer.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.morphologizer.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0

[components.morphologizer.model.tok2vec.pooling]
@layers = "reduce_mean.v1"

[components.ner]
factory = "ner"
moves = null
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.parser]
factory = "parser"
learn_tokens = false
min_action_freq = 30
moves = null
update_with_oracle_cut_size = 100

[components.parser.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "parser"
extra_state_tokens = false
hidden_width = 128
maxout_pieces = 3
use_upper = false
nO = null

[components.parser.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transformer"
max_batch_items = 4096
set_extra_annotations = {"@annotation_setters":"spacy-transformers.null_annotation_setter.v1"}

[components.transformer.model]
@architectures = "spacy-transformers.TransformerModel.v1"
name = "cl-tohoku/bert-base-japanese-v2"

[components.transformer.model.get_spans]
@span_getters = "spacy-transformers.strided_spans.v1"
window = 128
stride = 96

[components.transformer.model.tokenizer_config]
use_fast = false
mecab_kwargs = {"mecab_dic":"unidic_lite"}

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = \${paths.dev}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[corpora.train]
@readers = "spacy.Corpus.v1"
path = \${paths.train}
max_length = 500
gold_preproc = false
limit = 0
augmenter = null

[training]
accumulate_gradient = 3
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
seed = \${system.seed}
gpu_allocator = \${system.gpu_allocator}
dropout = 0.1
patience = 0
max_epochs = 0
max_steps = 20000
eval_frequency = 200
frozen_components = []
before_to_disk = null

[training.batcher]
@batchers = "spacy.batch_by_padded.v1"
discard_oversize = true
size = 2000
buffer = 256
get_length = null

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"

[training.optimizer]
@optimizers = "Adam.v1"
beta1 = 0.9
beta2 = 0.999
L2_is_weight_decay = true
L2 = 0.01
grad_clip = 1.0
use_averages = false
eps = 0.00000001

[training.optimizer.learn_rate]
@schedules = "warmup_linear.v1"
warmup_steps = 250
total_steps = 20000
initial_rate = 0.00005

[training.score_weights]
dep_las_per_type = null
sents_p = null
sents_r = null
ents_per_type = null
dep_uas = 0.17
dep_las = 0.17
sents_f = 0.0
ents_f = 0.33
ents_p = 0.0
ents_r = 0.0
tag_acc = 0.33

[pretraining]

[initialize]
vectors = null
init_tok2vec = \${paths.init_tok2vec}
vocab_data = null
lookups = null
before_init = null
after_init = null

[initialize.components]

[initialize.tokenizer]
EOF

In [7]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
# corpus/ja_gsd-ud-test.ne.spacy
# ja_gsd-ud-dev.ne.spacy	ja_gsd-ud-test.ne.spacy  ja_gsd-ud-train.ne.spacy

In [9]:
!python -m spacy train config.cfg --gpu-id 0 --output ./ja_ud_japanese_gsd_2.6_ne_trf \
  --paths.train ./corpus/ja_gsd-ud-train.ne.spacy --paths.dev ./corpus/ja_gsd-ud-dev.ne.spacy

✔ Created output directory: ja_ud_japanese_gsd_2.6_ne_trf
ℹ Saving to output directory: ja_ud_japanese_gsd_2.6_ne_trf
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-10-28 11:47:18,534] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-28 11:47:18,546] [INFO] Pipeline: ['transformer', 'parser', 'ner', 'morphologizer']
INFO:spacy:Pipeline: ['transformer', 'parser', 'ner', 'morphologizer']
[2022-10-28 11:47:18,553] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-28 11:47:18,554] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bia

In [10]:
!ls ja_ud_japanese_gsd_2.6_ne_trf/
# model-best  model-last

model-best  model-last


In [11]:
!mkdir -p ./package
!python -m spacy package ./ja_ud_japanese_gsd_2.6_ne_trf/model-best ./package --name udjagsd26ne_trf --version 0.0.1

ℹ Building package artifacts: sdist
✔ Including 1 package requirement(s) from meta and config
spacy-transformers>=1.1.8,<1.2.0
✔ Loaded meta.json from file
ja_ud_japanese_gsd_2.6_ne_trf/model-best/meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory 'ja_udjagsd26ne_trf-0.0.1'
package/ja_udjagsd26ne_trf-0.0.1
running sdist
running egg_info
creating ja_udjagsd26ne_trf.egg-info
writing ja_udjagsd26ne_trf.egg-info/PKG-INFO
writing dependency_links to ja_udjagsd26ne_trf.egg-info/dependency_links.txt
writing entry points to ja_udjagsd26ne_trf.egg-info/entry_points.txt
writing requirements to ja_udjagsd26ne_trf.egg-info/requires.txt
writing top-level names to ja_udjagsd26ne_trf.egg-info/top_level.txt
writing manifest file 'ja_udjagsd26ne_trf.egg-info/SOURCES.txt'
reading manifest file 'ja_udjagsd26ne_trf.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'ja_udjagsd26ne_trf.egg-info/SOURCES.txt'
running check


creating ja_ud

In [12]:
from google.colab import files
files.download("./package/ja_udjagsd26ne_trf-0.0.1/dist/ja_udjagsd26ne_trf-0.0.1.tar.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
!pip install ./package/ja_udjagsd26ne_trf-0.0.1

!pip list | grep udjagsd26ne
# ja-udjagsd26ne-trf            0.0.1 

import spacy
nlp = spacy.load("ja_udjagsd26ne_trf")
nlp.pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./package/ja_udjagsd26ne_trf-0.0.1
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ja-udjagsd26ne-trf: filename=ja_udjagsd26ne_trf-0.0.1-py3-none-any.whl size=414357934 sha256=0cb4fbec8068f6d33af90b776cc7026ee1ac360ed25ac5b51cfd36cabc5e6436
  Stored in directory: /root/.cache/pip/wheels/85/21/6d/080ae8c645c01f015b236e2a8d8b4ea68a9965d9db8eff3f25
Successfully built ja-udjagsd26ne-trf
ja-udjagsd26ne-trf            0.0.1


[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x7fd8b0b02830>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7fd746ac2d50>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fd746ac29d0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7fd739c11f30>)]

In [14]:
nlp2 = spacy.load("./ja_ud_japanese_gsd_2.6_ne_trf/model-best")

In [15]:
doc = nlp2("""
闇金業者に借金をしてしまうと、元金の返済はおろか、借金が増えていく一方で、心苦しい思いをしてしまうものです。
そして、そのような状態になってしまうと、知りつつ借金したから誰にも頼れない心情となり、一人で悩みを抱え込み、解決策に気づけなくなってしまうことも多々あるのです。
そのような際の解決方法を見つけたいなら、まず他人に相談するのが大切ではないでしょうか。
一人で悩みを抱え込むと、周りが見えなくなってしまい、余計につらくなってしまうことがあります。
そして、そのような時には弁護士や司法書士に相談するのが最良の対処法といえ、無料の相談サービスを利用して悩みを聞いてもらうのがよいのではないでしょうか。
こういった法務事務所・法律事務所ではいつも、ヤミ金問題に困っている人たちの相談に乗っていて、弁護士や司法書士からの助言がもらえるので、とても役立ちます。
ヤミ金解決に数多くの実績がある、人情派弁護士に、あなたの悩み・苦しみを、まずは無料相談してみては？
北海道から沖縄まで、全国対応の事務所だから、東温市に住んでいる方ももちろん相談OKです。
解決に伴う手数料も、分割払いOKなので、今は支払うお金が無くても、安心して相談できます。
ヤミ金に悩んでいる東温市の女性の方、今スグここで無料相談してみませんか？
地元の東温市の近くの法務事務所・法律事務所は下記を参考にどうぞ。
一般の司法書士・弁護士事務所は、通常の債務整理についての相談は受け付けているのですが、闇金の問題に関しては、相談を受け付けていないケースも多くあります。
東温市近辺の弁護士事務所や司法書士事務所でも、そういうところが多いかもしれません。
闇金業者から借金をしてしまった結果、最初こそ普通の督促の電話があったものの後半には脅迫染みた電話がかかってきたり、家に怒鳴り込まれたりするなどあくどい債権回収を行われる場合があります。
このような状況になってしまったら、弁護士や司法書士等に相談することが一番ですが、借金をしていることへの負い目や、闇金業者からの仕返しを恐れて相談や通報をためらってしまう場合もあるようです。
また、弁護士や司法書士に依頼するのに歯止めをかけてしまう理由に、報酬を支払う余裕がない、という場合もあります。
法律関連に強いのが彼らですから、闇金業者が如何に不法かを分かりやすく説明し、現在の目も当てられない状況を解決する手伝いしてくれます。
弁護士や司法書士への依頼費用が高いと悩んでいる方も、無料で相談を受け付けている司法書士事務所や弁護士事務所もあります。
ヤミ金業者からお金を借りてしまうと、元金の返済どころか、借金が増えていくばかりで、ドツボにはまっていってしまうものです。
しかも、闇金にお金を借りるという状況は、ずいぶんお金に追い詰められた状況だけに、そのような人が、ヤミ金業者から借金をして返済できるわけがありません。
そして、そのような状態になってしまうと、知りつつ借りたから人に頼れない心情になり、1人だけで悩みを抱え込んで、打開方法に気づけなくなってしまうこともあります。
そのような時の対策というのは、まず人に相談することが一番です。
""")


In [16]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


一人 99 101 QUANTITY
一人 181 183 QUANTITY
北海道 431 434 GPE
沖縄 436 438 GPE
東温市 453 456 GPE
東温市 530 533 GPE
東温市 561 564 GPE
東温市 668 671 GPE
司法書士事務所 681 688 ORG
司法書士 824 828 TITLE_AFFIX
一番 837 839 ORDINAL
司法書士事務所 1060 1067 ORG
1人 1258 1260 QUANTITY
一番 1323 1325 ORDINAL


In [17]:
doc.ents

(一人, 一人, 北海道, 沖縄, 東温市, 東温市, 東温市, 東温市, 司法書士事務所, 司法書士, 一番, 司法書士事務所, 1人, 一番)

In [18]:
len(doc._.trf_data.tokens["input_texts"])

9

In [19]:
doc._.trf_data.width

768